In [ ]:
using RCall
using DataFrames, DataFramesMeta
using CSV
using DataVoyager
ENV["LINES"] = 10
ENV["COLUMNS"]=10000;

### Load Covid Data

source: https://covid.ourworldindata.org/data/owid-covid-data.csv

In [ ]:
df = CSV.read("owid-covid-data-latest.csv",DataFrame)

### Check Data Summary

In [ ]:
R"summary($df)"

In [ ]:
ENV["LINES"] = 10
describe(df)

### Column Subset: Remove redundant columns

In [ ]:
features = [:continent,:location,:date,:new_cases,:new_deaths,:population, :population_density]
dfa = df[:,features]
dfn = @chain dfa begin
    @subset(:new_cases .>= 0)
    @subset(:new_deaths .>= 0)
end
describe(dfn)

### Row Subset: Filter  Asia

In [ ]:
dfasia = R"""
          options(warn=-1)
          library(tidyverse)
          $dfn %>% filter(continent == "Asia") %>% 
          filter(new_cases >= 0) %>% filter(new_deaths >= 0)
        """ |> rcopy

In [ ]:
dfasia=@subset(dfn, :continent .== "Asia")

### Visualize Asia Covid Trend

In [ ]:
# dfasia |> Voyager()

### Zoom-in to Philippines

In [ ]:
dfph=R"$dfasia %>% filter(location=='Philippines')" |> rcopy

In [ ]:
# dfph |> Voyager()

In [ ]:
dfcases = R"$dfasia %>% select(location,date,new_cases,new_deaths)" |> rcopy

### All Asia

In [ ]:
# dfcases |> Voyager()

### Plot interesting trends from the visualization

In [ ]:
RCall.rcall_p(:options, rcalljl_options=Dict(:width => 500, :height => 10000))
R"""
options(warn=-1)
library(ggplot2)

ggplot($dfcases,aes(date,new_cases))+
  geom_point(aes(color=new_deaths))+ylim(0,10000)+
  facet_grid(location ~ .)
"""

In [ ]:
RCall.rcall_p(:options, rcalljl_options=Dict(:width => 700, :height => 300))

function plotdeaths(df::DataFrame,ctry::String)
  R"""
     options(warn=-1)
     library(ggplot2)
     ctr = $df %>% filter(location == $ctry)
     ggplot(ctr,aes(date,new_deaths))+
        geom_point(aes(color=new_deaths))+
        scale_x_date(date_breaks = "1 month", date_labels =  "%b %Y")+
        theme(axis.text.x=element_text(angle=60, hjust=1))
  """
end

function plotcases(df::DataFrame,ctry::String)
  R"""
     options(warn=-1)
     library(ggplot2)
     ctr = $df %>% filter(location == $ctry)
     ggplot(ctr,aes(date,new_cases))+
        geom_point(aes(color=new_cases))+
        scale_x_date(date_breaks = "1 month", date_labels =  "%b %Y")+
        theme(axis.text.x=element_text(angle=60, hjust=1))
  """
end
function covidplot(df,ctry)
    display(plotcases(df,ctry));
    display(plotdeaths(df,ctry));
end

In [ ]:
covidplot(df,"Philippines")

In [ ]:
covidplot(df,"United Kingdom")

In [ ]:
covidplot(df,"Ireland")

In [ ]:
covidplot(df,"Israel")

In [ ]:
covidplot(df,"India")

In [ ]:
covidplot(df,"Thailand")

In [ ]:
covidplot(df,"Indonesia")

In [ ]:
covidplot(df,"Malaysia")

In [ ]:
covidplot(df,"China")

In [ ]:
covidplot(df,"Australia")

In [ ]:
covidplot(df,"Russia")

In [ ]:
covidplot(df,"Netherlands")

In [ ]:
covidplot(df,"Sweden")

In [ ]:
covidplot(df,"Brazil")

In [ ]:
covidplot(df,"United States")

In [ ]:
covidplot(df,"Argentina")